In [ ]:
import plotly.express as px
from plotly_express import data, scatter_mapbox
from PIL import Image

elif menu == 'Cause des feux':

    # TITRES ET INTRODUCTION

    st.title("Causes des incendies")
    st.markdown("---")
    st.subheader(" Présentation des données")
    st.title("En fonction des années")
    st.write('')
    st.write("La taille des ronds prend en compte la surface brûlée en km2 et la couleur correspond à la cause d'origine de l'incendie.")
    st.write("*En déplaçant le curseur vous pourrez remarquer que les feux sont présents sur l'ensemble du territoire Américain, et pourtant, "
                 "ils ne sont pas visibles sur la carte. En effet, la plupart des incendies n'ont eu qu'un impact très léger en terme de surface "
                 "brûlée. Les bulles visibles ci-dessous attestent la présence d'incendies aux dégâts conséquents.*")

    # MAP1 1992-2003
    
    st.write('**Carte des incendies aux Etats-Unis de 1992 à 2003**')

    dfcause1 = perfect[perfect["fire_year"] <= 2003]
    df_sorted_year1 = dfcause1.sort_values(by = 'fire_year')

    df_geo = data.carshare()
    fig_cause1 = px.scatter_mapbox(df_geo,
                      lat= df_sorted_year1.latitude,
                      lon= df_sorted_year1.longitude,
                      color= df_sorted_year1.stat_cause_descr,
                      color_discrete_sequence = ['lightgreen', 'blue', 'green', 'grey', 'black', 'red',
                                                   'yellow', 'darkblue', 'darkorange', 'lightblue', 'pink',
                                                   'cyan', 'magenta'],
                      size=df_sorted_year1.size_km2,
                      size_max=35,
                      zoom=2,
                      animation_frame = df_sorted_year1.fire_year,
                      mapbox_style="stamen-terrain") #

    st.plotly_chart(fig_cause1, use_container_width=True)

    st.write('')
    st.write('**Carte des incendies aux Etats-Unis de 2004 à 2015**')

    # MAP2 2004-2015

    dfcause2 = perfect[perfect["fire_year"] > 2003]
    df_sorted_year2 = dfcause2.sort_values(by='fire_year')

    fig_cause2 = px.scatter_mapbox(df_geo,
                             lat=df_sorted_year2.latitude,
                             lon=df_sorted_year2.longitude,
                             color=df_sorted_year2.stat_cause_descr,
                             color_discrete_sequence=['grey', 'blue', 'yellow', 'red', 'lightgreen', 'black',
                                                      'green', 'darkorange', 'darkblue', 'magenta', 'lightblue',
                                                      'cyan', 'pink'],
                             size=df_sorted_year2.size_km2,
                             size_max=35,
                             zoom=2,
                             animation_frame=df_sorted_year2.fire_year,
                             mapbox_style="stamen-terrain")  #

    st.plotly_chart(fig_cause2, use_container_width=True)

    # Partie cause / mois (A VERIFIER SI ON MET OU PAS CAR PEU COHERENT)
    st.title("En fonction des mois")

    perfect_sorted = perfect.sort_values(by='fire_year')
    figviolon = px.violin(perfect_sorted,
                    x='discovery_doy', y="stat_cause_descr",
                    points=False,
                    color='stat_cause_descr',
                    color_discrete_sequence=['blue', 'darkorange', 'green', 'grey', 'black', 'red',
                                             'yellow', 'darkblue', 'lightgreen',
                                             'lightblue', 'pink', 'cyan', 'magenta'],
                    range_x=[0, 366],
                    animation_frame=perfect_sorted['fire_year'],
                    orientation='h',
    )


    st.plotly_chart(figviolon, use_container_width=True)

    # Ecriture des titres Machine Learning
    st.title("Machine learning : Prédire la cause de l'incendie")

    image = Image.open("image_Mage.png")

    st.image(image, width = 50)

    st.write("Est-il possible de prédire la cause d'un incendie de manière générale ?")

    # MACHINE LEARNING
    from sklearn import tree, preprocessing
    import sklearn.ensemble as ske
    from sklearn.model_selection import train_test_split

    dfMLCause = pd.read_csv('DATA_ORIGINAL.csv')

    dfML = dfMLCause[["fire_year", "discovery_date", "stat_cause_descr", "state"]]

    dfML['stat_cause_descr'] = dfML['stat_cause_descr'].replace(
        ['Miscellaneous', 'Lightning', 'Debris Burning', 'Campfire',
         'Equipment Use', 'Arson', 'Children', 'Railroad',
         'Smoking',
         'Powerline', 'Structure', 'Fireworks',
         'Missing/Undefined'],
        ['Divers', 'Eclair', 'Debris brulant', 'Feu de camp',
         "Utilisation d'équipement", 'Criminel', 'Enfant',
         'Chemin de fer', 'Cigarette',
         'Ligne électrique', 'Structure', "Feu d'artifice",
         'Non identifié'])

    le = preprocessing.LabelEncoder()

    dfML['state'] = le.fit_transform(dfML['state'])
    dfML['discovery_date'] = le.fit_transform(dfML['discovery_date'])

    def set_label(cat):
        cause = 0
        natural = ['Eclair']
        accidental = ['Structure', "Feu d'artifice", 'Ligne électrique', 'Chemin de fer', 'Cigarette',
                      'Enfant', 'Feu de camp', "Utilisation d'équipement", 'Debris brulant']
        malicious = ['Criminel']
        other = ['Non identifié', 'Divers']
        if cat in natural:
            cause = 1
        elif cat in accidental:
            cause = 2
        elif cat in malicious:
            cause = 3
        else:
            cause = 4
        return cause

    dfML['label'] = dfML['stat_cause_descr'].apply(lambda x: set_label(x))
    dfML = dfML.drop('stat_cause_descr', axis=1)

    X = dfML.drop(['label'], axis=1).values
    y = dfML['label'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    rf_labels = ske.RandomForestClassifier(n_estimators=200)
    rf_labels = rf_labels.fit(X_train, y_train)
    Score = rf_labels.score(X_test, y_test)
    if st.button('Score'):
        st.write(Score)

    st.write("Est-il possible de prédire les incendies d'origines criminelles ?")